In [ ]:
import numpy as np
import sys
import tensorflow as tf
from keras.losses import Loss
from sklearn.preprocessing import StandardScaler
from scipy.sparse import csr_matrix
import pandas as pd
import datetime
import time
import io
from sklearn.model_selection import train_test_split

In [ ]:
import torch
import typing

from pathlib import Path

from torchvision import datasets

from sklearn import model_selection


def lambda_top_func(mu, k, y, alpha):
    m = y.shape[0]
    r = (y[k:] - mu).sum(dim=0)
    return mu + r.div(m * (alpha + 1) - k)


def lambda_bottom_func(mu, k, y, alpha):
    m = y.shape[0]
    r = (y[:k] - mu).sum(dim=0)
    return mu + r.div(m * alpha + k)


def alpha_fn(pi, lambda_):
    return (pi * lambda_) ** -1 + 1.0 - lambda_ ** -1


def beta_fn(pi, lambda_):
    return lambda_ * (pi) ** -1 + 1.0 - lambda_


def policy_risk(pi, y1, y0):
    return (pi * y1 + (1 - pi) * y0).mean()

def complete_propensity(x, u, gamma, beta=0.75):
    logit = beta * x + 0.5
    nominal = (1 + np.exp(-logit)) ** -1
    alpha = alpha_fn(nominal, gamma)
    beta = beta_fn(nominal, gamma)
    return (u / alpha) + ((1 - u) / beta)


def f_mu(x, t, u, theta=4.0):
    mu = (
        (2 * t - 1) * x
        + (2.0 * t - 1)
        - 2 * np.sin((4 * t - 2) * x)
        - (theta * u - 2) * (1 + 0.5 * x)
    )
    return mu


def linear_normalization(x, new_min, new_max):
    return (x - x.min()) * (new_max - new_min) / (x.max() - x.min()) + new_min


class HCMNIST(datasets.MNIST):
    def __init__(
        self,
        root: str,
        gamma_star: float,
        split: str = "train",
        mode: str = "mu",
        p_u: str = "bernoulli",
        theta: float = 4.0,
        beta: float = 0.75,
        sigma_y: float = 1.0,
        domain: float = 2.0,
        seed: int = 1331,
        transform: typing.Optional[typing.Callable] = None,
        target_transform: typing.Optional[typing.Callable] = None,
        download: bool = True,
    ) -> None:
        train = split == "train" or split == "valid"
        root = Path.home() / "quince_datasets" if root is None else Path(root)
        self.__class__.__name__ = "MNIST"
        super(HCMNIST, self).__init__(
            root,
            train=train,
            transform=transform,
            target_transform=target_transform,
            download=download,
        )
        self.data = self.data.view(len(self.targets), -1).numpy()
        self.targets = self.targets.numpy()

        if train:
            (
                data_train,
                data_valid,
                targets_train,
                targets_valid,
            ) = model_selection.train_test_split(
                self.data, self.targets, test_size=0.3, random_state=seed
            )
            self.data = data_train if split == "train" else data_valid
            self.targets = targets_train if split == "train" else targets_valid

        self.mode = mode
        self.dim_input = [1, 28, 28]
        self.dim_treatment = 1
        self.dim_output = 1

        self.phi_model = fit_phi_model(
            root=root, edges=torch.arange(-domain, domain + 0.1, (2 * domain) / 10),
        )

        size = (self.__len__(), 1)
        rng = np.random.RandomState(seed=seed)
        if p_u == "bernoulli":
            self.u = rng.binomial(1, 0.5, size=size).astype("float32")
        elif p_u == "uniform":
            self.u = rng.uniform(size=size).astype("float32")
        elif p_u == "beta_bi":
            self.u = rng.beta(0.5, 0.5, size=size).astype("float32")
        elif p_u == "beta_uni":
            self.u = rng.beta(2, 5, size=size).astype("float32")
        else:
            raise NotImplementedError(f"{p_u} is not a supported distribution")

        phi = self.phi
        self.pi = (
            complete_propensity(x=phi, u=self.u, gamma=gamma_star, beta=beta)
            .astype("float32")
            .ravel()
        )
        self.t = rng.binomial(1, self.pi).astype("float32")
        eps = (sigma_y * rng.normal(size=self.t.shape)).astype("float32")
        self.mu0 = (
            f_mu(x=phi, t=0.0, u=self.u, theta=theta).astype("float32").ravel()
        )
        self.mu1 = (
            f_mu(x=phi, t=1.0, u=self.u, theta=theta).astype("float32").ravel()
        )
        self.y0 = self.mu0 + eps
        self.y1 = self.mu1 + eps
        self.y = self.t * self.y1 + (1 - self.t) * self.y0
        self.tau = self.mu1 - self.mu0
        self.y_mean = np.array([0.0], dtype="float32")
        self.y_std = np.array([1.0], dtype="float32")

    def __getitem__(self, index):
        x = ((self.data[index].astype("float32") / 255.0) - 0.1307) / 0.3081
        t = self.t[index : index + 1]
        if self.mode == "pi":
            return x, t
        elif self.mode == "mu":
            return np.hstack([x, t]), self.y[index : index + 1]
        else:
            raise NotImplementedError(
                f"{self.mode} not supported. Choose from 'pi'  for propensity models or 'mu' for expected outcome models"
            )

    @property
    def phi(self):
        x = ((self.data.astype("float32") / 255.0) - 0.1307) / 0.3081
        z = np.zeros_like(self.targets.astype("float32"))
        for k, v in self.phi_model.items():
            ind = self.targets == k
            x_ind = x[ind].reshape(ind.sum(), -1)
            means = x_ind.mean(axis=-1)
            z[ind] = linear_normalization(
                np.clip((means - v["mu"]) / v["sigma"], -1.4, 1.4), v["lo"], v["hi"]
            )
        return np.expand_dims(z, -1)

    @property
    def x(self):
        return ((self.data.astype("float32") / 255.0) - 0.1307) / 0.3081


def fit_phi_model(root, edges):
    ds = datasets.MNIST(root=root)
    data = (ds.data.float().div(255) - 0.1307).div(0.3081).view(len(ds), -1)
    model = {}
    digits = torch.unique(ds.targets)
    for i, digit in enumerate(digits):
        lo, hi = edges[i : i + 2]
        ind = ds.targets == digit
        data_ind = data[ind].view(ind.sum(), -1)
        means = data_ind.mean(dim=-1)
        mu = means.mean()
        sigma = means.std()
        model.update(
            {
                digit.item(): {
                    "mu": mu.item(),
                    "sigma": sigma.item(),
                    "lo": lo.item(),
                    "hi": hi.item(),
                }
            }
        )
    return model

In [ ]:
def get_HMINIST_data(sim_num):
  obj = HCMNIST(root = "./",gamma_star = 1.1, seed=sim_num)
  # Assuming X is a matrix of size (42000, 784)
  # Assuming indices is the array of generated indices
  indices = np.random.choice(42000, size=3000, replace=False)
  # Select subrows from X using the generated indices

  X = obj.x[indices]
  Y_factual = obj.y[indices]
  T = obj.t[indices]
  mu_0 = obj.mu0[indices]
  mu_1 = obj.mu1[indices]

  X_train, X_test, Y_factual_train, _, T_train, _, mu_0_train, mu_0_test, mu_1_train, mu_1_test = train_test_split(X, Y_factual, T, mu_0, mu_1, test_size=0.0000001, random_state=sim_num)

  X_train = X_train.astype('float32')
  Y_factual_train = Y_factual_train.astype('float32') #most GPUs only compute 32-bit floats
  T_train = T_train.astype('float32')
  mu_0_train = mu_0_train.astype('float32')
  mu_1_train = mu_1_train.astype('float32')

  X_test = X_test.astype('float32')
  mu_0_test = mu_0_test.astype('float32')
  mu_1_test = mu_1_test.astype('float32')

  data_train={'x':X_train,'y':Y_factual_train,'t':T_train,'mu_0':mu_0_train,'mu_1':mu_1_train}
  data_train['t']=data_train['t'].reshape(-1,1) #we're just padding one dimensional vectors with an additional dimension
  data_train['y']=data_train['y'].reshape(-1,1)
  data_train['mu_0']=data_train['mu_0'].reshape(-1,1) #we're just padding one dimensional vectors with an additional dimension
  data_train['mu_1']=data_train['mu_1'].reshape(-1,1)

  #rescaling y between 0 and 1 often makes training of DL regressors easier
  data_train['y_scaler'] = StandardScaler().fit(data_train['y'])
  data_train['ys'] = data_train['y_scaler'].transform(data_train['y'])

  data_test={'x':X_test,'mu_0':mu_0_test,'mu_1':mu_1_test}
  data_test['y_scaler'] = data_train['y_scaler']
  data_test['mu_0']=data_test['mu_0'].reshape(-1,1) #we're just padding one dimensional vectors with an additional dimension
  data_test['mu_1']=data_test['mu_1'].reshape(-1,1)

  return data_train, data_test

In [ ]:
def pdist2sq(x,y):
    x2 = tf.reduce_sum(x ** 2, axis=-1, keepdims=True)
    y2 = tf.reduce_sum(y ** 2, axis=-1, keepdims=True)
    dist = x2 + tf.transpose(y2, (1, 0)) - 2. * x @ tf.transpose(y, (1, 0))
    return dist

class CFRNet_Loss(Loss):
  #initialize instance attributes
  def __init__(self, alpha=1.,sigma=1.):
      super().__init__()
      self.alpha = alpha # balances regression loss and MMD IPM
      self.rbf_sigma=sigma #for gaussian kernel
      self.name='cfrnet_loss'

  def split_pred(self,concat_pred):
      #generic helper to make sure we dont make mistakes
      preds={}
      preds['y0_pred'] = concat_pred[:, 0]
      preds['y1_pred'] = concat_pred[:, 1]
      preds['phi'] = concat_pred[:, 2:]
      return preds

  def rbf_kernel(self, x, y):
    return tf.exp(-pdist2sq(x,y)/tf.square(self.rbf_sigma))

  def calc_mmdsq(self, Phi, t):
    Phic, Phit =tf.dynamic_partition(Phi,tf.cast(tf.squeeze(t),tf.int32),2)

    Kcc = self.rbf_kernel(Phic,Phic)
    Kct = self.rbf_kernel(Phic,Phit)
    Ktt = self.rbf_kernel(Phit,Phit)

    m = tf.cast(tf.shape(Phic)[0],Phi.dtype)
    n = tf.cast(tf.shape(Phit)[0],Phi.dtype)

    mmd = 1.0/(m*(m-1.0))*(tf.reduce_sum(Kcc))
    mmd = mmd + 1.0/(n*(n-1.0))*(tf.reduce_sum(Ktt))
    mmd = mmd - 2.0/(m*n)*tf.reduce_sum(Kct)
    return mmd * tf.ones_like(t)

  def mmdsq_loss(self, concat_true,concat_pred):
    t_true = concat_true[:, 1]
    p=self.split_pred(concat_pred)
    mmdsq_loss = tf.reduce_mean(self.calc_mmdsq(p['phi'],t_true))
    return mmdsq_loss

  def regression_loss(self,concat_true,concat_pred):
      y_true = concat_true[:, 0]
      t_true = concat_true[:, 1]
      p = self.split_pred(concat_pred)
      loss0 = tf.reduce_mean((1. - t_true) * tf.square(y_true - p['y0_pred']))
      loss1 = tf.reduce_mean(t_true * tf.square(y_true - p['y1_pred']))
      return loss0+loss1

  def cfr_loss(self,concat_true,concat_pred):
      lossR = self.regression_loss(concat_true,concat_pred)
      lossIPM = self.mmdsq_loss(concat_true,concat_pred)
      return lossR + self.alpha * lossIPM

      #return lossR + self.alpha * lossIPM

  #compute loss
  def call(self, concat_true, concat_pred):
      return self.cfr_loss(concat_true,concat_pred)

In [ ]:
import tensorflow as tf
import numpy as np

from keras.layers import Input
from keras.layers import Dense
from keras.layers import Concatenate
from keras import regularizers
from keras import Model
from keras.losses import binary_crossentropy
from keras.metrics import binary_accuracy
from keras.losses import Loss

def make_tarnet(input_dim, reg_l2):

    x = Input(shape=(input_dim,), name='input')

    # representation
    phi = Dense(units=512, activation='elu', kernel_initializer='RandomNormal',name='phi_1')(x)
    phi = Dense(units=256, activation='elu', kernel_initializer='RandomNormal',name='phi_2')(phi)

    # HYPOTHESIS
    y0_hidden = Dense(units=256, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_1')(phi)
    y1_hidden = Dense(units=128, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_1')(phi)

    # second layer
    y0_hidden = Dense(units=256, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y0_hidden_2')(y0_hidden)
    y1_hidden = Dense(units=128, activation='elu', kernel_regularizer=regularizers.l2(reg_l2),name='y1_hidden_2')(y1_hidden)

    # third
    y0_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y0_predictions')(y0_hidden)
    y1_predictions = Dense(units=1, activation=None, kernel_regularizer=regularizers.l2(reg_l2), name='y1_predictions')(y1_hidden)

    concat_pred = Concatenate(1)([y0_predictions, y1_predictions,phi])
    model = Model(inputs=x, outputs=concat_pred)

    return model

In [ ]:

#https://towardsdatascience.com/implementing-macro-f1-score-in-keras-what-not-to-do-e9f1aa04029d
class Eval_metrics_train():
    def __init__(self,data):
        self.data=data #feed the callback the full dataset
        #needed for PEHEnn; Called in self.find_ynn
        self.data['o_idx']=tf.range(self.data['t'].shape[0])
        self.data['c_idx']=self.data['o_idx'][self.data['t'].squeeze()==0] #These are the indices of the control units
        self.data['t_idx']=self.data['o_idx'][self.data['t'].squeeze()==1] #These are the indices of the treated units

    def split_pred(self,concat_pred):
        preds={}
        preds['y0_pred'] = self.data['y_scaler'].inverse_transform(concat_pred[:, 0].reshape(-1, 1))
        preds['y1_pred'] = self.data['y_scaler'].inverse_transform(concat_pred[:, 1].reshape(-1, 1))
        preds['t_pred'] = concat_pred[:, 2]
        preds['epsilon'] = concat_pred[:, 3]
        preds['phi'] = concat_pred[:, 4:]
        return preds

    def ATE_absolute_error(self,concat_pred):
        p = self.split_pred(concat_pred)
        ATT_pred = tf.gather(params=self.data['y'], indices=self.data['t_idx']) - tf.gather(params=p['y0_pred'], indices=self.data['t_idx'])
        ATU_pred = tf.gather(params=p['y1_pred'], indices=self.data['c_idx']) - tf.gather(params=self.data['y'], indices=self.data['c_idx'])
        ATE_pred = tf.reduce_mean(tf.concat([ATT_pred,ATU_pred], axis=0)) #stitch em back up!
        ATE_actual = tf.reduce_mean(self.data['mu_1']-self.data['mu_0'])
        return tf.abs(ATE_actual- ATE_pred)

    def ITE_RMSE_error(self,concat_pred):
        #simulation only
        p = self.split_pred(concat_pred)
        y_1_treated_group = tf.gather(params=self.data['y'], indices=self.data['t_idx'])
        y_0_treated_group = tf.gather(params=p['y0_pred'], indices=self.data['t_idx'])

        mu_1_treated_group = tf.gather(params=self.data['mu_1'], indices=self.data['t_idx'])
        mu_0_treated_group = tf.gather(params=self.data['mu_0'], indices=self.data['t_idx'])


        treat_grp_error = (y_1_treated_group - y_0_treated_group) - (mu_1_treated_group - mu_0_treated_group)

        y_1_control_group = tf.gather(params=p['y1_pred'], indices=self.data['c_idx'])
        y_0_control_group = tf.gather(params=self.data['y'], indices=self.data['c_idx'])

        mu_1_control_group = tf.gather(params=self.data['mu_1'], indices=self.data['c_idx'])
        mu_0_control_group = tf.gather(params=self.data['mu_0'], indices=self.data['c_idx'])

        control_grp_error = (y_1_control_group - y_0_control_group) - (mu_1_control_group - mu_0_control_group)


        ITE_error = tf.concat([treat_grp_error, control_grp_error], axis=0)
        ITE_RMSE_error = tf.sqrt(tf.reduce_mean(tf.square(ITE_error)))
        return ITE_RMSE_error

In [ ]:

#https://towardsdatascience.com/implementing-macro-f1-score-in-keras-what-not-to-do-e9f1aa04029d
class Eval_metrics_test():
    def __init__(self,data):
        self.data=data #feed the callback the full dataset

    def split_pred(self,concat_pred):
        preds={}
        preds['y0_pred'] = self.data['y_scaler'].inverse_transform(concat_pred[:, 0].reshape(-1, 1))
        preds['y1_pred'] = self.data['y_scaler'].inverse_transform(concat_pred[:, 1].reshape(-1, 1))
        preds['t_pred'] = concat_pred[:, 2]
        preds['epsilon'] = concat_pred[:, 3]
        preds['phi'] = concat_pred[:, 4:]
        return preds


    def PEHE(self,concat_pred):
        #simulation only
        p = self.split_pred(concat_pred)
        cate_err=tf.reduce_mean( tf.square( ( (self.data['mu_1']-self.data['mu_0']) - (p['y1_pred']-p['y0_pred']) ) ) )
        return tf.sqrt(cate_err)

    def ATE_absolute_error_outsample(self,concat_pred):
        #simulation only
        p = self.split_pred(concat_pred)
        ATE_actual = tf.reduce_mean(self.data['mu_1']-self.data['mu_0'])
        ATE_pred = tf.reduce_mean(p['y1_pred']-p['y0_pred'])
        return tf.abs(ATE_actual- ATE_pred)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau, TerminateOnNaN
from tensorflow.keras.optimizers import SGD, Adam
import io
from IPython.display import display, clear_output
#Colab command to allow us to run Colab in TF2
%load_ext tensorboard
# data=get_news_data(1)
val_split=0.20
batch_size=4000
verbose=1
i = 0
tf.random.set_seed(i)
np.random.seed(i)
!rm -rf ./logs_CFRnet/
sim_evals = []
for j in range(0,100):
    clear_output(wait=True)
    print(j+1)
    data_train, data_test = get_HMINIST_data(j+1)
    yt = np.concatenate([data_train['ys'], data_train['t']], 1)

    # Clear any logs from previous runs
    start_time = time.time()
    log_dir = "logs_CFRnet/fit/" +str(j)+"/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    file_writer = tf.summary.create_file_writer(log_dir + "/metrics")
    file_writer.set_as_default()
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)

    adam_callbacks = [
        TerminateOnNaN(),
        EarlyStopping(monitor='val_loss', patience=30, min_delta=0.),
        ReduceLROnPlateau(monitor='loss', factor=0.5, patience=10, verbose=verbose, mode='auto',
                          min_delta=1e-8, cooldown=0, min_lr=0),
        tensorboard_callback
    ]


    cfrnet_model=make_tarnet(data_train['x'].shape[1],.01)
    cfrnet_loss=CFRNet_Loss(alpha=1.0)


    cfrnet_model.compile(optimizer=Adam(learning_rate=0.0025),
                         loss=cfrnet_loss,
                         metrics=[cfrnet_loss,cfrnet_loss.regression_loss,cfrnet_loss.mmdsq_loss])

    cfrnet_model.fit(x=data_train['x'],y=yt,
                     callbacks=adam_callbacks,
                     validation_split=val_split,
                     epochs=10000,
                     batch_size=batch_size,
                     verbose=verbose)

    end_time = time.time()

    elapsed_time_seconds = end_time - start_time
    # Convert elapsed time to minutes
    elapsed_time_minutes = elapsed_time_seconds / 60

    Evaluation_metrics_insample = Eval_metrics_train(data_train)
    concat_pred_insample = cfrnet_model.predict(data_train['x'])
    ATE_abs_insample = Evaluation_metrics_insample.ATE_absolute_error(concat_pred_insample)
    ITE_RMSE_insample = Evaluation_metrics_insample.ITE_RMSE_error(concat_pred_insample)

    Evaluation_metrics_outsample = Eval_metrics_test(data_test)
    concat_pred_outsample = cfrnet_model.predict(data_test['x'])
    ATE_abs_outsample = Evaluation_metrics_outsample.ATE_absolute_error_outsample(concat_pred_outsample)
    PEHE_outsample = Evaluation_metrics_outsample.PEHE(concat_pred_outsample)

    metrics = [ATE_abs_insample.numpy(), ITE_RMSE_insample.numpy(), ATE_abs_outsample.numpy(), PEHE_outsample.numpy(), elapsed_time_minutes]
    sim_evals.append(metrics)

4
Epoch 1/10000
1/1 [==============================] - 1s 1s/step - loss: 11.3635 - cfrnet_loss: 1.9965 - regression_loss: 1.9948 - mmdsq_loss: 0.0017 - val_loss: 94.9982 - val_cfrnet_loss: 85.7236 - val_regression_loss: 85.7167 - val_mmdsq_loss: 0.0069 - lr: 0.0025
Epoch 2/10000
1/1 [==============================] - 0s 38ms/step - loss: 94.8754 - cfrnet_loss: 85.6008 - regression_loss: 85.5991 - mmdsq_loss: 0.0017 - val_loss: 17.7154 - val_cfrnet_loss: 8.4632 - val_regression_loss: 8.4563 - val_mmdsq_loss: 0.0069 - lr: 0.0025
Epoch 3/10000
1/1 [==============================] - 0s 36ms/step - loss: 17.3137 - cfrnet_loss: 8.0614 - regression_loss: 8.0597 - mmdsq_loss: 0.0017 - val_loss: 11.2425 - val_cfrnet_loss: 2.0052 - val_regression_loss: 1.9984 - val_mmdsq_loss: 0.0069 - lr: 0.0025
Epoch 4/10000
1/1 [==============================] - 0s 35ms/step - loss: 11.2320 - cfrnet_loss: 1.9947 - regression_loss: 1.9930 - mmdsq_loss: 0.0017 - val_loss: 17.4352 - val_cfrnet_loss: 8.2081 - va

Epoch 32/10000
1/1 [==============================] - 0s 34ms/step - loss: 9.5423 - cfrnet_loss: 0.8276 - regression_loss: 0.8259 - mmdsq_loss: 0.0017 - val_loss: 9.5551 - val_cfrnet_loss: 0.8681 - val_regression_loss: 0.8612 - val_mmdsq_loss: 0.0069 - lr: 0.0025
Epoch 33/10000
1/1 [==============================] - 0s 36ms/step - loss: 9.4281 - cfrnet_loss: 0.7411 - regression_loss: 0.7394 - mmdsq_loss: 0.0017 - val_loss: 9.4908 - val_cfrnet_loss: 0.8318 - val_regression_loss: 0.8250 - val_mmdsq_loss: 0.0068 - lr: 0.0025
Epoch 34/10000
1/1 [==============================] - 0s 34ms/step - loss: 9.3664 - cfrnet_loss: 0.7074 - regression_loss: 0.7056 - mmdsq_loss: 0.0017 - val_loss: 9.4994 - val_cfrnet_loss: 0.8688 - val_regression_loss: 0.8618 - val_mmdsq_loss: 0.0069 - lr: 0.0025
Epoch 35/10000
1/1 [==============================] - 0s 35ms/step - loss: 9.3789 - cfrnet_loss: 0.7483 - regression_loss: 0.7465 - mmdsq_loss: 0.0017 - val_loss: 9.4850 - val_cfrnet_loss: 0.8830 - val_regres

1/1 [==============================] - 0s 35ms/step - loss: 8.3658 - cfrnet_loss: 0.5947 - regression_loss: 0.5930 - mmdsq_loss: 0.0017 - val_loss: 8.5802 - val_cfrnet_loss: 0.8404 - val_regression_loss: 0.8335 - val_mmdsq_loss: 0.0069 - lr: 0.0025
Epoch 64/10000
1/1 [==============================] - 0s 36ms/step - loss: 8.3343 - cfrnet_loss: 0.5945 - regression_loss: 0.5928 - mmdsq_loss: 0.0017 - val_loss: 8.5462 - val_cfrnet_loss: 0.8377 - val_regression_loss: 0.8308 - val_mmdsq_loss: 0.0069 - lr: 0.0025
Epoch 65/10000
1/1 [==============================] - 0s 36ms/step - loss: 8.2966 - cfrnet_loss: 0.5881 - regression_loss: 0.5864 - mmdsq_loss: 0.0017 - val_loss: 8.5141 - val_cfrnet_loss: 0.8369 - val_regression_loss: 0.8300 - val_mmdsq_loss: 0.0069 - lr: 0.0025
Epoch 66/10000
1/1 [==============================] - 0s 35ms/step - loss: 8.2599 - cfrnet_loss: 0.5827 - regression_loss: 0.5810 - mmdsq_loss: 0.0017 - val_loss: 8.4877 - val_cfrnet_loss: 0.8418 - val_regression_loss: 0.83

1/1 [==============================] - 0s 36ms/step - loss: 7.7741 - cfrnet_loss: 0.9444 - regression_loss: 0.9427 - mmdsq_loss: 0.0017 - val_loss: 7.6405 - val_cfrnet_loss: 0.8395 - val_regression_loss: 0.8326 - val_mmdsq_loss: 0.0069 - lr: 0.0025
Epoch 95/10000
1/1 [==============================] - 0s 36ms/step - loss: 7.3463 - cfrnet_loss: 0.5454 - regression_loss: 0.5437 - mmdsq_loss: 0.0017 - val_loss: 7.8429 - val_cfrnet_loss: 1.0704 - val_regression_loss: 1.0635 - val_mmdsq_loss: 0.0069 - lr: 0.0025
Epoch 96/10000
1/1 [==============================] - 0s 36ms/step - loss: 7.6034 - cfrnet_loss: 0.8308 - regression_loss: 0.8291 - mmdsq_loss: 0.0017 - val_loss: 7.6308 - val_cfrnet_loss: 0.8863 - val_regression_loss: 0.8795 - val_mmdsq_loss: 0.0069 - lr: 0.0025
Epoch 97/10000
1/1 [==============================] - 0s 36ms/step - loss: 7.3814 - cfrnet_loss: 0.6370 - regression_loss: 0.6352 - mmdsq_loss: 0.0017 - val_loss: 7.6211 - val_cfrnet_loss: 0.9045 - val_regression_loss: 0.89

Epoch 125/10000
1/1 [==============================] - 0s 36ms/step - loss: 6.4729 - cfrnet_loss: 0.4725 - regression_loss: 0.4708 - mmdsq_loss: 0.0017 - val_loss: 6.8468 - val_cfrnet_loss: 0.8715 - val_regression_loss: 0.8646 - val_mmdsq_loss: 0.0069 - lr: 0.0025
Epoch 126/10000
1/1 [==============================] - 0s 34ms/step - loss: 6.4376 - cfrnet_loss: 0.4623 - regression_loss: 0.4606 - mmdsq_loss: 0.0017 - val_loss: 6.8294 - val_cfrnet_loss: 0.8791 - val_regression_loss: 0.8722 - val_mmdsq_loss: 0.0069 - lr: 0.0025
Epoch 127/10000
1/1 [==============================] - 0s 38ms/step - loss: 6.4143 - cfrnet_loss: 0.4641 - regression_loss: 0.4624 - mmdsq_loss: 0.0017 - val_loss: 6.8055 - val_cfrnet_loss: 0.8802 - val_regression_loss: 0.8734 - val_mmdsq_loss: 0.0068 - lr: 0.0025
Epoch 128/10000
1/1 [==============================] - 0s 36ms/step - loss: 6.3809 - cfrnet_loss: 0.4556 - regression_loss: 0.4539 - mmdsq_loss: 0.0017 - val_loss: 6.7869 - val_cfrnet_loss: 0.8864 - val_re

...

1/1 [==============================] - 0s 35ms/step - loss: 2.4397 - cfrnet_loss: 0.0453 - regression_loss: 0.0436 - mmdsq_loss: 0.0017 - val_loss: 3.6457 - val_cfrnet_loss: 1.2543 - val_regression_loss: 1.2474 - val_mmdsq_loss: 0.0069 - lr: 6.2500e-04
Epoch 550/10000
1/1 [==============================] - 0s 32ms/step - loss: 2.4365 - cfrnet_loss: 0.0451 - regression_loss: 0.0434 - mmdsq_loss: 0.0017 - val_loss: 3.6431 - val_cfrnet_loss: 1.2547 - val_regression_loss: 1.2478 - val_mmdsq_loss: 0.0069 - lr: 6.2500e-04
Epoch 551/10000
1/1 [==============================] - 0s 35ms/step - loss: 2.4333 - cfrnet_loss: 0.0449 - regression_loss: 0.0431 - mmdsq_loss: 0.0017 - val_loss: 3.6406 - val_cfrnet_loss: 1.2551 - val_regression_loss: 1.2482 - val_mmdsq_loss: 0.0069 - lr: 6.2500e-04
Epoch 552/10000
1/1 [==============================] - 0s 37ms/step - loss: 2.4301 - cfrnet_loss: 0.0446 - regression_loss: 0.0429 - mmdsq_loss: 0.0017 - val_loss: 3.6383 - val_cfrnet_loss: 1.2558 - val_regres

1/1 [==============================] - 0s 37ms/step - loss: 2.3444 - cfrnet_loss: 0.0384 - regression_loss: 0.0367 - mmdsq_loss: 0.0017 - val_loss: 3.5715 - val_cfrnet_loss: 1.2685 - val_regression_loss: 1.2616 - val_mmdsq_loss: 0.0069 - lr: 6.2500e-04
Epoch 580/10000
1/1 [==============================] - 0s 34ms/step - loss: 2.3413 - cfrnet_loss: 0.0382 - regression_loss: 0.0365 - mmdsq_loss: 0.0017 - val_loss: 3.5690 - val_cfrnet_loss: 1.2689 - val_regression_loss: 1.2620 - val_mmdsq_loss: 0.0069 - lr: 6.2500e-04
Epoch 581/10000
1/1 [==============================] - 0s 36ms/step - loss: 2.3382 - cfrnet_loss: 0.0380 - regression_loss: 0.0363 - mmdsq_loss: 0.0017 - val_loss: 3.5665 - val_cfrnet_loss: 1.2693 - val_regression_loss: 1.2624 - val_mmdsq_loss: 0.0069 - lr: 6.2500e-04
Epoch 582/10000
1/1 [==============================] - 0s 37ms/step - loss: 2.3350 - cfrnet_loss: 0.0378 - regression_loss: 0.0361 - mmdsq_loss: 0.0017 - val_loss: 3.5642 - val_cfrnet_loss: 1.2698 - val_regres

1/1 [==============================] - 0s 36ms/step - loss: 2.2521 - cfrnet_loss: 0.0326 - regression_loss: 0.0309 - mmdsq_loss: 0.0017 - val_loss: 3.4983 - val_cfrnet_loss: 1.2817 - val_regression_loss: 1.2748 - val_mmdsq_loss: 0.0069 - lr: 6.2500e-04
Epoch 610/10000
1/1 [==============================] - 0s 37ms/step - loss: 2.2491 - cfrnet_loss: 0.0324 - regression_loss: 0.0307 - mmdsq_loss: 0.0017 - val_loss: 3.4960 - val_cfrnet_loss: 1.2822 - val_regression_loss: 1.2753 - val_mmdsq_loss: 0.0069 - lr: 6.2500e-04
Epoch 611/10000
1/1 [==============================] - 0s 37ms/step - loss: 2.2460 - cfrnet_loss: 0.0322 - regression_loss: 0.0305 - mmdsq_loss: 0.0017 - val_loss: 3.4935 - val_cfrnet_loss: 1.2826 - val_regression_loss: 1.2757 - val_mmdsq_loss: 0.0069 - lr: 6.2500e-04
Epoch 612/10000
1/1 [==============================] - 0s 34ms/step - loss: 2.2430 - cfrnet_loss: 0.0321 - regression_loss: 0.0303 - mmdsq_loss: 0.0017 - val_loss: 3.4911 - val_cfrnet_loss: 1.2830 - val_regres

1/1 [==============================] - 0s 38ms/step - loss: 2.1628 - cfrnet_loss: 0.0277 - regression_loss: 0.0260 - mmdsq_loss: 0.0017 - val_loss: 3.4268 - val_cfrnet_loss: 1.2945 - val_regression_loss: 1.2876 - val_mmdsq_loss: 0.0069 - lr: 6.2500e-04
Epoch 640/10000
1/1 [==============================] - 0s 35ms/step - loss: 2.1599 - cfrnet_loss: 0.0276 - regression_loss: 0.0258 - mmdsq_loss: 0.0017 - val_loss: 3.4243 - val_cfrnet_loss: 1.2948 - val_regression_loss: 1.2879 - val_mmdsq_loss: 0.0069 - lr: 6.2500e-04
Epoch 641/10000
1/1 [==============================] - 0s 35ms/step - loss: 2.1569 - cfrnet_loss: 0.0274 - regression_loss: 0.0257 - mmdsq_loss: 0.0017 - val_loss: 3.4218 - val_cfrnet_loss: 1.2950 - val_regression_loss: 1.2881 - val_mmdsq_loss: 0.0069 - lr: 6.2500e-04
Epoch 642/10000
1/1 [==============================] - 0s 33ms/step - loss: 2.1540 - cfrnet_loss: 0.0273 - regression_loss: 0.0255 - mmdsq_loss: 0.0017 - val_loss: 3.4195 - val_cfrnet_loss: 1.2955 - val_regres

1/1 [==============================] - 0s 34ms/step - loss: 2.0765 - cfrnet_loss: 0.0237 - regression_loss: 0.0219 - mmdsq_loss: 0.0017 - val_loss: 3.3558 - val_cfrnet_loss: 1.3057 - val_regression_loss: 1.2988 - val_mmdsq_loss: 0.0069 - lr: 6.2500e-04
Epoch 670/10000
1/1 [==============================] - 0s 36ms/step - loss: 2.0736 - cfrnet_loss: 0.0235 - regression_loss: 0.0218 - mmdsq_loss: 0.0017 - val_loss: 3.3537 - val_cfrnet_loss: 1.3063 - val_regression_loss: 1.2994 - val_mmdsq_loss: 0.0069 - lr: 6.2500e-04
Epoch 671/10000
1/1 [==============================] - 0s 36ms/step - loss: 2.0708 - cfrnet_loss: 0.0234 - regression_loss: 0.0217 - mmdsq_loss: 0.0017 - val_loss: 3.3514 - val_cfrnet_loss: 1.3068 - val_regression_loss: 1.2998 - val_mmdsq_loss: 0.0069 - lr: 6.2500e-04
Epoch 672/10000
1/1 [==============================] - 0s 35ms/step - loss: 2.0680 - cfrnet_loss: 0.0233 - regression_loss: 0.0216 - mmdsq_loss: 0.0017 - val_loss: 3.3489 - val_cfrnet_loss: 1.3069 - val_regres

In [ ]:
sim_evals = np.asarray(sim_evals)
# with open('eval_metrics_news_20.npy', 'wb') as f:
#     np.save(f,sim_evals)

In [ ]:
print(" Insample ATE error for CFR-net(mean over 50) =", round(np.mean(sim_evals[:,0]),2),"+-",round((np.std(sim_evals[:,0], ddof=1) / np.sqrt(np.size(sim_evals[:,0]))),2))
print(" Insample ITE_RMSE error for CFR-net(mean over 50) =", round(np.mean(sim_evals[:,1]),2),"+-",round((np.std(sim_evals[:,1], ddof=1) / np.sqrt(np.size(sim_evals[:,1]))),2))
print(" Outsample ATE error for CFR-net(mean over 50) =", round(np.mean(sim_evals[:,2]),2),"+-",round((np.std(sim_evals[:,2], ddof=1) / np.sqrt(np.size(sim_evals[:,2]))),2))
print(" Outsample PEHE error for CFR-net(mean over 50) =", round(np.mean(sim_evals[:,3]),2),"+-",round((np.std(sim_evals[:,3], ddof=1) / np.sqrt(np.size(sim_evals[:,3]))),2))
print(" Wall time for CFR-net(sum over 50) =", np.sum(sim_evals[:,4]))

In [ ]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()